## 3. BUILD THE NEURAL NETWORK
https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Neural network는 많은 layers/modules로 이루어져 data에 대해서 연산을 한다. torch.nn은 사용자들만의 neural network를 만드는데에 필요한 building block들을 제공해준다.


이후 예제에서는 FashionMNIST 이미지들을 분류하는 neural network를 build해 본다.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision  import datasets, transforms # torchvision 관련해서는 https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html 참고

`-` Training을 위한 Device 설정하기

가능하다면, GPU나 MPS와 같은 hardware accelerator에서 모델을 학습하고자 한다. 먼저 GPU나 MPS가 사용가능한지 보자.
- hardware accelerator: hardware acceleration은 컴퓨팅에서 일부 기능을 CPU에서 구동하는 방식보다 빠르게 수행할 수 있는 하드웨어(GPU)의 사용을 말한다.

- MPS: MPS(Multi-Process Service)는 다수의 프로세스가 동시에 단일 GPU에서 실행되도록 해주는 런타임 서비스다.

In [3]:
device = ("cuda" if torch.cuda.is_available()
         else "mps"
         if torch.backends.mps.is_available()
         else "cpu")
print(f"Using {device} device")

Using cuda device


`-` Class 정의하기

여기서 사용자는 nn.Module을 subclassing 함으로써 Neural Network를 정의할 수 있다. 그리고 neural network layers를 __init__를 통해서 initialize할 수 있다. 모든 nn.Module subclass는 forward method를 통해서 input 데이터에 대한 연산을 구현한다.
- Subclassing 출처: https://velog.io/@yhlee9753/Java-%EC%9D%98-Generics

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            return logits

이제 실제 input을 넣어 output이 나오는 동작이 되는 모델 class의 instance를 만들고, 모델의 구조를 print한다.

In [5]:
model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 레이어들

여기서 입력인 image는 아래와 같이 정의하고
위의 `NeuralNetwork()`의 레이어를 작동과정과 함께 하나씩 살펴보자.

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


`-` nn.Flatten

위의 nn.Flatten는 2D인 28 by 28 image를 contiguose한 784길이의 array로 변환해준다. (단, batch dimension은 유지된다.)

In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


`-` nn.Linear

nn.Linear는 nn.Linear에 저장된 weights와 bias 값을 가지고 input에 linear transformation을 적용한다.
아래는 위에서 784길이로 변환한 input에 weights곱과 bias를 더하여 다음 layer에 20개의 output을 전달한다.

In [10]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20) # 
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


`-` nn.ReLU

Non-linear 한 activation function인 ReLU이다. Linear가 linear transformation을 한 다음 여기에 nonlinearity를 적용해주는 역할을 한다. 이는 모델이 다양한 output을 내주도록 도와준다. 즉, 모델의 표현력이 증가한다.  
- ReLU는 0미만 값은 0으로, 0이상의 값은 identity한 값이 나오도록 mapping해주는 함수.
- ReLU이외에도 다른 non linear activation functions가 존재함.

In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3809, -0.5069, -0.6069,  0.3530, -0.1545,  0.2423,  0.0947, -0.0172,
         -0.4334, -0.2419,  0.2784,  0.4531,  0.1922,  0.2756,  0.3507,  0.5308,
         -0.3168,  0.3161, -0.2114, -0.2160],
        [-0.6083, -0.2409, -0.5716,  0.4486, -0.0911,  0.2253, -0.1714, -0.1431,
         -0.3146, -0.4192,  0.5556,  0.1647,  0.4398,  0.5658,  0.1044,  0.7514,
         -0.1698,  0.0641,  0.1544, -0.2806],
        [-0.4601, -0.5548, -0.1492,  0.6013,  0.3029,  0.4519, -0.0502, -0.1066,
         -0.4079, -0.2639,  0.3063,  0.2510,  0.4217,  0.7640,  0.0640,  0.3840,
         -0.3652,  0.1182,  0.1693, -0.4177]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.3530, 0.0000, 0.2423, 0.0947, 0.0000, 0.0000,
         0.0000, 0.2784, 0.4531, 0.1922, 0.2756, 0.3507, 0.5308, 0.0000, 0.3161,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4486, 0.0000, 0.2253, 0.0000, 0.0000, 0.0000,
         0.0000, 0.5556, 0.1647, 0.4398, 0.5658, 0.10

`-` nn.Sequential

nn.Sequential은 모듈들을 담아, 담긴 순서대로 적용해주는 container 역할을 한다.
예를들어, 아래의 nn.Sequential에 input을 넣으면 flatten, layer1, ReLU, Linear 순서로 input이 처리된다.  
- 모듈 예: nn.Flatten, nn.Linear, nn.ReLU, ...

nn.Sequential 모듈의 장점은, 위에서 flatten = nn.Flatten(), layer1 = nn.Linear() 와 같이 일일이 지정하던 것을 nn.Sequential이라는 모듈하나에 담아, 빠르게 network구성을 할 수 있다는 점이다.

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

`-` nn.Softmax

마지막의 linear layer에서 나온 값의 범위를 [-infty, infty]에서 [0,1]로 변환해주는 역할을 한다. 이는 모델의 각 라벨에 대한 예측 확률로써 볼수도 있다.  
- 여기서 dim 인자는 어떤 차원으로 softmax를 적용할지를 결정한다.

아래 예시는 NeuralNetwork 모델의 출력값인 logits에 softmax를 적용하는 예시이다.

In [17]:
softmax = nn.Softmax(dim=1) # dim=1로 Softmax를 정의하는 함수를 정의
pred_probab = softmax(logits) # 위에서 정의한 Softmax를 neural net의 output에 적용
print(pred_probab)

tensor([[0.0914, 0.0946, 0.1138, 0.0845, 0.1091, 0.1213, 0.0949, 0.0790, 0.1435,
         0.0678],
        [0.0949, 0.1163, 0.1055, 0.0772, 0.0987, 0.1056, 0.1103, 0.0756, 0.1462,
         0.0697],
        [0.0944, 0.1123, 0.1030, 0.0742, 0.1047, 0.1154, 0.0966, 0.0802, 0.1543,
         0.0648]], grad_fn=<SoftmaxBackward0>)


## 모델 파라메터

Neural net안의 layer들에 파라메터들이 있다. 이들은 training과정에서 optimized되는데, nn.Module이 모든 파라메터들에 access할 수 있게 해준다 한다.  
parameter에 접근하는 방법은  모델의 parameters() 혹은 named_parameters() methods를 사용하는 것이다.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0344,  0.0256,  0.0130,  ..., -0.0339,  0.0349, -0.0023],
        [-0.0035,  0.0305,  0.0306,  ...,  0.0169,  0.0114, -0.0106]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0134,  0.0203], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0063, -0.0081,  0.0038,  ...,  0.0171, -0.0084,  0.0218],
        [ 0.0184,  0.0190, -0.0368,  ...,  0.0164, -0.0380, -0.0320]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 